In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Bipolar_disorder"
cohort = "GSE45484"

# Input paths
in_trait_dir = "../../input/GEO/Bipolar_disorder"
in_cohort_dir = "../../input/GEO/Bipolar_disorder/GSE45484"

# Output paths
out_data_file = "../../output/preprocess/Bipolar_disorder/GSE45484.csv"
out_gene_data_file = "../../output/preprocess/Bipolar_disorder/gene_data/GSE45484.csv"
out_clinical_data_file = "../../output/preprocess/Bipolar_disorder/clinical_data/GSE45484.csv"
json_path = "../../output/preprocess/Bipolar_disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene-expression differences in peripheral blood between lithium responders and non-responders in the “Lithium Treatment -Moderate dose Use Study” (LiTMUS)"
!Series_summary	"Analysis of gene-expression changes in treatment responders vs non-responders to two different treatments among subjectrs participating in LiTMUS."
!Series_summary	"Results provide information on pathways that may be involved in the clinical response to Lithium in patients with bipolar disorder."
!Series_overall_design	"Total RNA isolated from PAXgene blood RNA tubes from 60 subjects with bipolar disorder, randomized to 2 treatment groups (OPT, Li+OPT) at 2 time-points (baseline, 1 month after treatment)"
Sample Characteristics Dictionary:
{0: ['treatment group: OPT', 'treatment group: Li+OPT'], 1: ['time point: baseline', 'time point: 1 month'], 2: ['responder: NO', 'responder: YES'], 3: ['sex: M', 'sex: F'], 4: ['age: 46', 'age: 44', 'age: 59', 'age: 32', 'age: 45', 'age: 25'

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
is_gene_available = True  # Based on the series title and summary, this dataset contains gene expression data

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
trait_row = 2  # 'responder: NO', 'responder: YES' - this represents bipolar disorder response to treatment
age_row = 4    # Contains age information with multiple values
gender_row = 3  # Contains sex information 'M' and 'F'

# 2.2 Data Type Conversion
def convert_trait(value):
    if value is None:
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary: YES=1 (responder), NO=0 (non-responder)
    if value.upper() == "YES":
        return 1
    elif value.upper() == "NO":
        return 0
    else:
        return None

def convert_age(value):
    if value is None:
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to integer
    try:
        return int(value)
    except:
        return None

def convert_gender(value):
    if value is None:
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary: F=0, M=1
    if value.upper() == "F":
        return 0
    elif value.upper() == "M":
        return 1
    else:
        return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (only if trait_row is not None)
if trait_row is not None:
    # Get the clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the resulting dataframe
    preview = preview_df(selected_clinical_df)
    print("Clinical Data Preview:")
    print(preview)
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Data Preview:
{'GSM1105438': [0.0, 46.0, 1.0], 'GSM1105439': [0.0, 44.0, 0.0], 'GSM1105440': [0.0, 46.0, 1.0], 'GSM1105441': [0.0, 44.0, 0.0], 'GSM1105442': [0.0, 59.0, 1.0], 'GSM1105443': [0.0, 32.0, 0.0], 'GSM1105444': [0.0, 59.0, 1.0], 'GSM1105445': [0.0, 32.0, 0.0], 'GSM1105446': [0.0, 45.0, 0.0], 'GSM1105447': [0.0, 25.0, 0.0], 'GSM1105448': [0.0, 45.0, 0.0], 'GSM1105449': [0.0, 25.0, 0.0], 'GSM1105450': [1.0, 25.0, 0.0], 'GSM1105451': [1.0, 26.0, 0.0], 'GSM1105452': [1.0, 25.0, 0.0], 'GSM1105453': [1.0, 26.0, 0.0], 'GSM1105454': [1.0, 43.0, 1.0], 'GSM1105455': [0.0, 24.0, 0.0], 'GSM1105456': [1.0, 43.0, 1.0], 'GSM1105457': [0.0, 24.0, 0.0], 'GSM1105458': [0.0, 43.0, 1.0], 'GSM1105459': [0.0, 43.0, 0.0], 'GSM1105460': [0.0, 43.0, 1.0], 'GSM1105461': [0.0, 43.0, 0.0], 'GSM1105462': [0.0, 38.0, 0.0], 'GSM1105463': [0.0, 47.0, 1.0], 'GSM1105464': [0.0, 38.0, 0.0], 'GSM1105465': [0.0, 47.0, 1.0], 'GSM1105466': [1.0, 37.0, 1.0], 'GSM1105467': [0.0, 57.0, 0.0], 'GSM1105468': [1

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Bipolar_disorder/GSE45484/GSE45484_series_matrix.txt.gz


Gene data shape: (47323, 120)
First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers start with "ILMN_" which indicates they are Illumina BeadArray probe IDs
# They are not human gene symbols and require mapping to standard gene symbols
# Illumina probe IDs are specific to the microarray platform and need to be converted

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Check if there are any columns that might contain gene information
sample_row = gene_annotation.iloc[0].to_dict()
print("\nFirst row as dictionary:")
for col, value in sample_row.items():
    print(f"{col}: {value}")

# Check if IDs in gene_data match IDs in annotation
print("\nComparing gene data IDs with annotation IDs:")
print("First 5 gene data IDs:", gene_data.index[:5].tolist())
print("First 5 annotation IDs:", gene_annotation['ID'].head().tolist())

# Properly check for exact ID matches between gene data and annotation
gene_data_ids = set(gene_data.index)
annotation_ids = set(gene_annotation['ID'].astype(str))
matching_ids = gene_data_ids.intersection(annotation_ids)
id_match_percentage = len(matching_ids) / len(gene_data_ids) * 100 if len(gene_data_ids) > 0 else 0

print(f"\nExact ID match between gene data and annotation:")
print(f"Matching IDs: {len(matching_ids)} out of {len(gene_data_ids)} ({id_match_percentage:.2f}%)")

# Check which columns might contain gene symbols for mapping
potential_gene_symbol_cols = [col for col in gene_annotation.columns 
                             if any(term in col.upper() for term in ['GENE', 'SYMBOL', 'NAME'])]
print(f"\nPotential columns for gene symbols: {potential_gene_symbol_cols}")

# Check if the identified columns contain non-null values
for col in potential_gene_symbol_cols:
    non_null_count = gene_annotation[col].notnull().sum()
    non_null_percent = non_null_count / len(gene_annotation) * 100
    print(f"Column '{col}': {non_null_count} non-null values ({non_null_percent:.2f}%)")



Gene annotation preview:
Columns in gene annotation: ['ID', 'Species', 'Source', 'Search_Key', 'Transcript', 'ILMN_Gene', 'Source_Reference_ID', 'RefSeq_ID', 'Unigene_ID', 'Entrez_Gene_ID', 'GI', 'Accession', 'Symbol', 'Protein_Product', 'Probe_Id', 'Array_Address_Id', 'Probe_Type', 'Probe_Start', 'SEQUENCE', 'Chromosome', 'Probe_Chr_Orientation', 'Probe_Coordinates', 'Cytoband', 'Definition', 'Ontology_Component', 'Ontology_Process', 'Ontology_Function', 'Synonyms', 'Obsolete_Probe_Id', 'GB_ACC']
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], '


Exact ID match between gene data and annotation:
Matching IDs: 47323 out of 47323 (100.00%)

Potential columns for gene symbols: ['ILMN_Gene', 'Unigene_ID', 'Entrez_Gene_ID', 'Symbol']
Column 'ILMN_Gene': 47323 non-null values (0.83%)
Column 'Unigene_ID': 3270 non-null values (0.06%)
Column 'Entrez_Gene_ID': 43960 non-null values (0.77%)
Column 'Symbol': 44837 non-null values (0.78%)


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns to use for probe IDs and gene symbols
# Based on previous step output, we need:
# - The 'ID' column (contains probe IDs like ILMN_1343048) that matches gene_data index
# - The 'Symbol' column (contains gene symbols) has 78% non-null values

# 2. Get gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')
print(f"Created mapping dataframe with shape: {mapping_df.shape}")
print(f"Sample of mapping data:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=mapping_df)
print(f"Created gene expression dataframe with shape: {gene_data.shape}")
print(f"First 5 genes and their expression values:")
print(gene_data.head())

# Save the processed gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Created mapping dataframe with shape: (44837, 2)
Sample of mapping data:
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB
Created gene expression dataframe with shape: (21464, 120)
First 5 genes and their expression values:
       GSM1105438  GSM1105439  GSM1105440  GSM1105441  GSM1105442  GSM1105443  \
Gene                                                                            
A1BG     14.03088    13.93148    13.81406    13.83391    13.78051    13.96313   
A1CF     20.32521    20.63063    20.66692    20.54370    20.64116    20.71162   
A26C3    20.39159    20.30175    20.52752    20.41063    20.47654    20.56898   
A2BP1    27.15309    27.27428    26.94768    27.12319    27.09179    26.95093   
A2LD1     7.32095     7.29238     7.47339     7.21109     7.26801     7.35792   

       GSM1

Gene expression data saved to ../../output/preprocess/Bipolar_disorder/gene_data/GSE45484.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
# Read the saved clinical data file
clinical_df = pd.read_csv(out_clinical_data_file)
print(f"Loaded clinical data shape: {clinical_df.shape}")

# Load proper clinical data with the correct conversion functions from Step 2
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,  # Using the previously defined function from Step 2
    age_row=age_row,
    convert_age=convert_age,      # Using the previously defined function from Step 2
    gender_row=gender_row,
    convert_gender=convert_gender # Using the previously defined function from Step 2
)

print(f"Selected clinical data shape: {selected_clinical_df.shape}")
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5] if not linked_data.empty else "Linked data is empty")

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Check for bias in features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from blood samples of bipolar disorder patients, analyzing response to lithium treatment."
)

# 6. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")

Gene data shape after normalization: (20259, 120)


Normalized gene expression data saved to ../../output/preprocess/Bipolar_disorder/gene_data/GSE45484.csv
Loaded clinical data shape: (3, 120)
Selected clinical data shape: (3, 120)
Clinical data preview:
{'GSM1105438': [0.0, 46.0, 1.0], 'GSM1105439': [0.0, 44.0, 0.0], 'GSM1105440': [0.0, 46.0, 1.0], 'GSM1105441': [0.0, 44.0, 0.0], 'GSM1105442': [0.0, 59.0, 1.0], 'GSM1105443': [0.0, 32.0, 0.0], 'GSM1105444': [0.0, 59.0, 1.0], 'GSM1105445': [0.0, 32.0, 0.0], 'GSM1105446': [0.0, 45.0, 0.0], 'GSM1105447': [0.0, 25.0, 0.0], 'GSM1105448': [0.0, 45.0, 0.0], 'GSM1105449': [0.0, 25.0, 0.0], 'GSM1105450': [1.0, 25.0, 0.0], 'GSM1105451': [1.0, 26.0, 0.0], 'GSM1105452': [1.0, 25.0, 0.0], 'GSM1105453': [1.0, 26.0, 0.0], 'GSM1105454': [1.0, 43.0, 1.0], 'GSM1105455': [0.0, 24.0, 0.0], 'GSM1105456': [1.0, 43.0, 1.0], 'GSM1105457': [0.0, 24.0, 0.0], 'GSM1105458': [0.0, 43.0, 1.0], 'GSM1105459': [0.0, 43.0, 0.0], 'GSM1105460': [0.0, 43.0, 1.0], 'GSM1105461': [0.0, 43.0, 0.0], 'GSM1105462': [0.0, 38.0, 0

Data shape after handling missing values: (120, 20262)
For the feature 'Bipolar_disorder', the least common label is '1.0' with 26 occurrences. This represents 21.67% of the dataset.
The distribution of the feature 'Bipolar_disorder' in this dataset is fine.

Quartiles for 'Age':
  25%: 28.5
  50% (Median): 39.5
  75%: 49.0
Min: 18.0
Max: 64.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 36 occurrences. This represents 30.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Bipolar_disorder/GSE45484.csv
